In [1]:
import requests

from portfoliobuilder import finnhub_endpoint, finnhub_key

### Notes for AAPL 2021 10-K
Year: response.json()['data'][0]['year']

Form: response.json()['data'][0]['form']

Report: response.json()['data'][0]['report']

Balance Sheet: response.json()['data'][0]['report']['bs']

Income Statement: response.json()['data'][0]['report']['ic']

Cash Flow Statement: response.json()['data'][0]['report']['cf']

Total Revenue: response.json()['data'][0]['report']['ic'][0]['value']

In [2]:
def get_fins_as_reported(symbol):
    url = finnhub_endpoint + 'stock/financials-reported'
    params = {'symbol': symbol, 'token': finnhub_key}
    return requests.get(url=url, params=params)


In [225]:
response = get_fins_as_reported('WFC')
r_json = response.json()

In [226]:
# r_json['data'][0]['report']['ic']

for i in r_json['data'][0]['report']['ic']:
    print(i['concept'])

NetIncomeLoss
OtherComprehensiveIncomeUnrealizedHoldingGainLossOnSecuritiesArisingDuringPeriodBeforeTax
wfc:OtherComprehensiveIncomeLossReclassificationAdjustmentForSaleOrWriteDownOfSecuritiesIncludedInNetIncomeBeforeTax
OtherComprehensiveIncomeUnrealizedGainLossOnDerivativesArisingDuringPeriodBeforeTax
OtherComprehensiveIncomeLossReclassificationAdjustmentFromAOCIOnDerivativesBeforeTax
OtherComprehensiveIncomeLossPensionAndOtherPostretirementBenefitPlansNetUnamortizedGainLossArisingDuringPeriodBeforeTax
wfc:OtherComprehensiveIncomeReclassificationOfDefinedBenefitPlansNetGainLossAmortizationOfNetPriorServiceCostCreditAndFinalizationOfValuationRecognizedInNetPeriodicBenefitCostBeforeTax
OtherComprehensiveIncomeForeignCurrencyTransactionAndTranslationGainLossBeforeReclassificationAndTax
OtherComprehensiveIncomeLossBeforeTax
OtherComprehensiveIncomeLossTax
OtherComprehensiveIncomeLossNetOfTax
OtherComprehensiveIncomeLossNetOfTaxPortionAttributableToNoncontrollingInterest
OtherComprehensiv